# Sigma_density: M Dwarfs with Spectroscopic Constraints

Calculating density constraints for M Dwarf KOIs with spectroscopic constraints on properties.

1. From spectroscopy, we know Temperature, Mass, and Radius.
2. Which isochrones (stellar models, with Temperature, Mass, and Radius as parameters) fit above star?
3. Now take into account GAIA distance for the star. From this, we know the Luminosity.
4. Look back at stellar models - which isochrone fits the Temperature, Mass Radius, and Luminority (from GAIA)?

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from pprint import pprint
import pandas as pd
from tqdm import tqdm
from astropy.table import Table

plt.rcParams['figure.figsize'] = [10, 5]

In [2]:
headers = ['KOI', 'KIC', 'Teff', 'ETeff', 'eTeff', 'Fe/H', 'e_Fe/H', 'M/H', 'e_M/H', 'SpType', 'Mstar', 'e_Mstar', 'Rstar', 'e_Rstar', 'Dis', 'Fe/H-T', 'e_Fe/H-T', 'Fe/H-M', 'e_Fe/H-M', 'M/H-M', 'e_M/H-M']

For now, removed any rows with missing data and saved them into "muirhead_data_no_missing_data.txt".

In [3]:
muirhead_data = pd.read_csv("/Users/sheilasagear/Dropbox (UFL)/Research/MetallicityProject/Data/Muirhead2013_isochrones/muirhead_data_no_missing_data.txt", sep=" ")
muirhead_data

,KOI,KIC,Teff,ETeff,eTeff,Fe/H,e_Fe/H,M/H,e_M/H,SpType,...,e_Mstar,Rstar,e_Rstar,Dis,Fe/H-T,e_Fe/H-T,Fe/H-M,e_Fe/H-M,M/H-M,e_M/H-M
0,4290,5868793,3187,62,88,0.10,0.16,0.07,0.13,M4V,...,0.04,0.20,0.03,77,-0.08,0.14,0.12,0.20,-0.02,0.21
1,2842,8733898,3204,40,67,-0.30,0.12,-0.21,0.12,M4V,...,0.02,0.18,0.02,34,-0.53,0.29,-0.30,0.11,-0.24,0.11
2,961,8561063,3204,59,46,-0.48,0.12,-0.33,0.12,M4V,...,0.05,0.17,0.04,38,-0.53,0.13,-0.64,0.14,-0.56,0.14
3,1725B,10905746,3253,62,57,0.16,0.13,0.11,0.12,M4V,...,0.04,0.23,0.04,34,0.15,0.12,0.21,0.14,0.10,0.14
4,2704,9730163,3267,69,51,0.27,0.13,0.19,0.12,M4V,...,0.05,0.26,0.04,118,0.22,0.14,0.38,0.15,0.14,0.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,1141,8346392,3968,391,69,-0.14,0.14,-0.10,0.13,M0V,...,0.07,0.55,0.06,230,-0.13,0.13,0.11,0.13,0.06,0.14
89,2057,9573685,3974,117,182,-0.05,0.15,-0.04,0.14,M0V,...,0.06,0.56,0.06,189,0.02,0.15,0.27,0.16,0.13,0.16
90,1085,10118816,3979,262,87,-0.33,0.13,-0.24,0.13,M0V,...,0.06,0.53,0.06,105,-0.31,0.14,-0.30,0.14,-0.15,0.13
91,531,10395543,4004,40,169,0.11,0.13,0.08,0.12,M0V,...,0.04,0.59,0.04,157,0.14,0.13,0.53,0.15,0.48,0.16


In [5]:
isochrones = pd.read_csv('/Users/sheilasagear/Dropbox (UFL)/Research/MetallicityProject/Data/Muirhead2013_isochrones/isochrones_sdss_spitzer_lowmass.dat', sep='\s\s+', engine='python')
isochrones

,mstar,logt,logl_ls,logg,radius,feh,afe,age,u,g,r,i,z,J,H,Ks,Kp,IRAC1,IRAC2
0,0.1609,3.523,-2.374,5.088,0.1898,-0.5,0.0,0.2,16.61,13.360,11.810,10.810,10.240,8.859,8.280,8.052,11.410,7.904,7.801
1,0.1709,3.523,-2.312,5.058,0.2026,-0.5,0.0,0.2,16.41,13.190,11.640,10.650,10.090,8.708,8.130,7.901,11.250,7.755,7.651
2,0.1810,3.525,-2.257,5.035,0.2140,-0.5,0.0,0.2,16.22,13.030,11.490,10.510,9.952,8.574,7.996,7.767,11.100,7.623,7.519
3,0.1911,3.526,-2.205,5.016,0.2249,-0.5,0.0,0.2,16.03,12.870,11.330,10.370,9.821,8.447,7.869,7.640,10.960,7.500,7.395
4,0.2011,3.528,-2.152,4.996,0.2359,-0.5,0.0,0.2,15.84,12.710,11.180,10.230,9.689,8.320,7.743,7.514,10.810,7.376,7.271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738474,0.6038,3.584,-1.155,4.662,0.6007,0.5,0.0,14.0,12.61,9.875,8.682,7.723,7.221,5.881,5.202,5.029,8.282,5.059,5.074
738475,0.6139,3.587,-1.125,4.653,0.6118,0.5,0.0,14.0,12.50,9.749,8.554,7.629,7.146,5.816,5.135,4.965,8.177,4.999,5.021
738476,0.6240,3.590,-1.095,4.644,0.6232,0.5,0.0,14.0,12.40,9.629,8.431,7.539,7.073,5.753,5.070,4.904,8.076,4.942,4.970
738477,0.6340,3.594,-1.066,4.635,0.6347,0.5,0.0,14.0,12.30,9.507,8.306,7.447,7.001,5.691,5.007,4.844,7.973,4.885,4.919


#### First, let's work with muirhead_data[0]: KIC 5868793.

1. Take Teff, Mass, and Radius.
2. Which isochrones fit these Teff, Mass, and Radius?

#### Reading Gaia-Kepler data (luminosities)

In [6]:
dat = Table.read('/Users/sheilasagear/Dropbox (UFL)/Research/MetallicityProject/Data/Kepler-Gaia/kepler_dr2_4arcsec.fits', format='fits')
df = dat.to_pandas()

/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'r_result_flag' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'r_modality_flag' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'teff_err1' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'teff_err2' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
muirhead_gaia = df[df['kepid'].isin(list(muirhead_data.KIC))]
muirhead_gaia = muirhead_gaia.reset_index()
muirhead_gaia.rename(columns={"index": "KIC"})

#muirhead_comb: planet hosts with spectroscopic data + Gaia/Kepler data in one table
muirhead_comb = pd.concat([muirhead_data, muirhead_gaia], axis=1)

In [8]:
def fit_isochrone(data, isochrones):
    """
       Inputs:
       data: pd.DataFrame. Spectroscopic data + Kepler/Gaia data for n stars in one table. (muirhead_comb)
       isochrones: pd.DataFrame. Isochrones table. (isochrones)
       
       Returns: list of pd.DataFrames. Each element of list is a pd.DataFrame of the isochrones that fit this star (index) BASED ONLY ON SPECTROSCOPY.
       """            
    
    iso_fits_final = list()
    
    #test each star in spectroscopy sample:
    #for i in tqdm(range(len(muirhead_comb))):
    for i in tqdm(range(1)):

        iso_fits = pd.DataFrame() 

        Teff_range = [data.Teff[i]-data.eTeff[i], data.Teff[i]+data.ETeff[i]]
        Mstar_range = [data.Mstar[i]-data.e_Mstar[i], data.Mstar[i]+data.e_Mstar[i]]
        Rstar_range = [data.Rstar[i]-data.e_Rstar[i], data.Rstar[i]+data.e_Rstar[i]]

        #test each stellar model to see if it falls within error bars:
        for j in range(len(isochrones)):
            if Teff_range[0] < 10**isochrones.logt[j] < Teff_range[1] and Mstar_range[0] < isochrones.mstar[j] < Mstar_range[1] and Rstar_range[0] < isochrones.radius[j] < Rstar_range[1]:
                iso_fits = iso_fits.append(isochrones.loc[[j]])

        iso_fits['KIC'] = muirhead_comb['KIC'][i]
        iso_fits['KOI'] = muirhead_comb['KOI'][i]

        iso_fits_final.append(iso_fits)

    return iso_fits_final


In [ ]:
isos = fit_isochrone(muirhead_comb, isochrones)

In [ ]:
isochrones['logg']

In [ ]:
#for i in range(len(isos)):
#    isos[i].to_csv("isochrone_fits/spectroscopy/iso_fits_" + str([i]) + ".csv")

In [10]:
#for i in range(12, len(isos)):
#    isos[i].to_csv("isochrone_fits/spectroscopy/iso_fits_" + str(isos[i]['KIC'].iloc[0]) + ".csv")

In [19]:
def fit_isochrone_lum(data, isochrones):
    """
       Inputs:
       data: pd.DataFrame. Spectroscopic data + Kepler/Gaia for n stars in one table. (muirhead_comb)
       isochrones: pd.DataFrame. Isochrones table. (isochrones)
       
       Returns: list of pd.DataFrames. Each element of list is a pd.DataFrame of the isochrones that fit this star (index) BASED ON SPECTROSCOPY AND GAIA LUMINOSITY.
       """    
    
    iso_fits_final = list()
    
    #for i in tqdm(range(len(muirhead_comb))):
    for i in range(1):
        
        iso_fits = pd.DataFrame() 
    
        Teff_range = [data.Teff[i]-data.eTeff[i], data.Teff[i]+data.ETeff[i]]
        Mstar_range = [data.Mstar[i]-data.e_Mstar[i], data.Mstar[i]+data.e_Mstar[i]]
        Rstar_range = [data.Rstar[i]-data.e_Rstar[i], data.Rstar[i]+data.e_Rstar[i]]
        lum_range = [data.lum_val[i]-data.lum_percentile_lower[i], data.lum_val[i]+data.lum_percentile_lower[i]]

        for j in range(len(isochrones)):
            if Teff_range[0] < 10**isochrones.logt[j] < Teff_range[1] and Mstar_range[0] < isochrones.mstar[j] < Mstar_range[1] and Rstar_range[0] < isochrones.radius[j] < Rstar_range[1] and lum_range[0] < 10**isochrones.logl_ls[j] < lum_range[1]:
                iso_fits = iso_fits.append(isochrones.loc[[j]])
        
        iso_fits['KIC'] = muirhead_comb['KIC'][i]
        iso_fits['KOI'] = muirhead_comb['KOI'][i]
        
        iso_fits_final.append(iso_fits)
            
    return iso_fits_final

In [20]:
iso_lums = fit_isochrone_lum(muirhead_comb, isochrones)


In [14]:
#for i in range(37, len(iso_lums)):
#    try:
#        iso_lums[i].to_csv("isochrone_fits/spectroscopy_luminosity/iso_lums_" + str(iso_lums[i]['KIC'].iloc[0]) + ".csv")
#    except IndexError:
#        pass

### Calculate rho

In [16]:
#Let's just do 1 star for now
ntargs = 1

In [1]:
def find_density_dist_symmetric(ntargs, masses, masserr1, masserr2, radii, raderr1, raderr2):
    
    rho_dist = np.zeros((ntargs, 1000))
    mass_dist = np.zeros((ntargs, 1000))
    rad_dist = np.zeros((ntargs, 1000))

    #star: indexing star
    #point: indexing PDF point for star
    
    for star in tqdm(range(ntargs)):
        
        rho_temp = np.zeros(1200)
        mass_temp = np.zeros(1200)
        rad_temp = np.zeros(1200)
        
        #####
        mass_temp = np.random.normal(masses[star], np.mean((abs(masserr2[star]), abs(masserr1[star]))), 1200)
        #len 1200
        rad_temp = np.random.normal(radii[star], np.mean((abs(raderr2[star]), abs(raderr1[star]))), 1200)
        #len 1200
        #####
        
        #for j from 0 to 1200
        #for each point in individual star PDF
        #Adding each density point to rho_temp (specific to this star)
        for point in range(len(mass_temp)):
            #if mass_dist[point] >= 0. and rad_dist[point] >= 0:
            if True:
                rho_temp[point] = density(mass_temp[point], rad_temp[point], sol_density)
                
        #Now rho_temp is a n-long array with this star. We want it to be 1000-long exactly
                                 
        while len(rho_temp) > 1000:
            temp_ind = np.random.randint(0, len(rho_temp)-1)
            rho_temp = np.delete(rho_temp, temp_ind)
            mass_temp = np.delete(mass_temp, temp_ind)
            rad_temp = np.delete(rad_temp, temp_ind)
        else:
            rho_dist[star] = rho_temp
            mass_dist[star] = mass_temp
            rad_dist[star] = rad_temp
    
    return rho_dist, mass_dist, rad_dist

In [3]:
def density(mass, radius, sol_density):
    """Mass in solar density
    Radius in solar density
    sol_density in kg/m^3"""

    return ((mass*1.989e30)/((4./3.)*np.pi*radius**3*696.34e6**3))/sol_density

In [ ]:
rho, mass, radius = find_density_dist_symmetric(ntargs, mass_dwarf, masserr1_dwarf, masserr2_dwarf, rad_dwarf, raderr1_dwarf, raderr2_dwarf, logg_dwarf)